In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
andrewmvd_car_plate_detection_path = kagglehub.dataset_download('andrewmvd/car-plate-detection')
# aslanahmedov_number_plate_detection_path = kagglehub.dataset_download('aslanahmedov/number-plate-detection')

print('Data source import complete.')


/Users/mokshitjain/Projects/SiddhCoder/Python/anpr/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mokshitjain/Projects/SiddhCoder/Python/anpr/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data source import complete.


### Load trained model

In [3]:
import torch
import cv2 as cv

yolo = torch.hub.load('ultralytics/yolov5', 'custom', path='./yolov5/runs/train/exp/weights/best.pt')

Using cache found in /Users/mokshitjain/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-22 Python-3.9.6 torch-2.5.1 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


### Inference

In [4]:
img = './image.png'

# Inference
results = yolo(img)

cordinates = results.xyxy[0][:, :-1]
results.pandas().xyxy[0]

/Users/mokshitjain/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


,xmin,ymin,xmax,ymax,confidence,class,name
0,186.281677,180.001419,340.038269,268.683594,0.793952,0,car_lisence_plate


## 4. Extract Number Plate

### Install EasyOCR

In [5]:
# %pip install easyocr -qq

### Read Plate Number

In [6]:
def read_plate_number(results, frame, reader):
    n = len(results)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    for i in range(n):
        row = cordinates[i] # Iterate through each image
        if row[4] >= 0.5: ## Take img with 0.5 confidence
            xmin, ymin, xmax, ymax = row[:4]
            plate = frame[int(ymin):int(ymax), int(xmin):int(xmax)]

            ## Preprocess Plate
            gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
            blurred = cv.bilateralFilter(gray, 17, 15, 15)
            # ret, thresh = cv.threshold(blurred, 125, 255, cv.THRESH_BINARY)

            ## OCR
            text = reader.readtext(blurred)
            text = ' '.join([t[1] for t in text])

            
            return text

            # return cv.cvtColor(final_img, cv.COLOR_BGR2RGB)
            # cv.imwrite(f'./labeled_img_{i}.jpg', cv.cvtColor(final_img, cv.COLOR_BGR2RGB))

In [7]:
import matplotlib.pyplot as plt
import easyocr
%matplotlib inline

reader = easyocr.Reader(['en'])
# cordinates = cordinates.cpu().numpy()

frame = cv.imread(img) ### reading the image
frame = cv.cvtColor(frame,cv.COLOR_BGR2RGB)


result = read_plate_number(cordinates, frame, reader)
print(result)

MH 14 TCF 200 F
